In [15]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################

from gait_scheduler import GaitScheduler
from env.quadruped.env import Env

from scipy.spatial.transform import Rotation
from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=False, base_fix=False)
env.reset()

array([ 0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.1,
       -0.7,  1.5,  0.1, -0.7,  1.5, -0.1, -0.7,  1.5,  0.1, -0.7,  1.5,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ])

In [40]:
vel_cmd = np.zeros(3)
vel_cmd[0] = 0.2
vel_cmd[1] = 0.0

ang_vel_cmd = np.zeros(3)
ang_vel_cmd[2] = 0.0

height_cmd = 0.35

In [41]:
time_step = 0.05
time_horizon = 20

max_accel = 0.1
max_ang_accel = 0.1

In [35]:
target_pos_list = []
target_vel_list = []
target_rpy_list = []
target_ang_vel_list = []

pos_com = np.array(env.model.com_pos)
pos_com[2] = height_cmd
vel_base = np.array(env.model.base_vel)
vel_base[2] = 0.0
rpy_base = np.array(env.model.base_rpy)
rpy_base[:2] = 0.0
ang_vel_base = np.array(env.model.base_ang_vel)
ang_vel_base[:2] = 0.0

vel_reach_t = np.linalg.norm(vel_cmd - vel_base)/max_accel
ang_vel_reach_t = np.linalg.norm(ang_vel_cmd - ang_vel_base)/max_ang_accel
target_accel = max_accel*(vel_cmd - vel_base)/(np.linalg.norm(vel_cmd - vel_base) + 1e-10)
target_ang_accel = max_ang_accel*(ang_vel_cmd - ang_vel_base)/(np.linalg.norm(ang_vel_cmd - ang_vel_base) + 1e-10)

target_base_vel = deepcopy(vel_base)
target_ang_vel = deepcopy(ang_vel_base)
target_pos = deepcopy(pos_com)
target_rpy = deepcopy(rpy_base)

for i in range(time_horizon):
    elapsed_t = time_step*(i + 1)
    if elapsed_t > vel_reach_t:
        target_base_vel = vel_cmd
    else:
        target_base_vel += time_step*target_accel
    if elapsed_t > ang_vel_reach_t:
        target_ang_vel = ang_vel_cmd
    else:
        target_ang_vel += time_step*target_ang_accel
        
    target_rpy += target_ang_vel*time_step
    r = Rotation.from_rotvec(target_rpy)
    target_vel = np.matmul(r.as_matrix(), target_base_vel)
    target_pos += target_vel*time_step

    target_vel_list.append(deepcopy(target_vel))
    target_pos_list.append(deepcopy(target_pos))
    target_rpy_list.append(deepcopy(target_rpy))
    target_ang_vel_list.append(deepcopy(target_ang_vel))